# Baseline Model 2
### Taking the last 20 actions as recommendations
### Kaggle Recall 0,46409

### 1. Setup

In [ ]:
! pip install tqdm

In [2]:
import polars as pl
import pandas as pd
from tqdm import tqdm

In [10]:
### Try to speed up process - did not work ###
#pip install -r requirements.txt --use-pep517
#! pip install cudf
# RAPIDS
#import cudf
#train = cudf.read_parquet("train.parquet")
#test = cudf.read_parquet("test.parquet")

In [3]:
# loading data
#train = pl.read_parquet('../data/train.parquet')
test = pl.read_parquet('../data/test.parquet')

In [4]:
print(f"Test shape {test.shape} ")

Test shape (6928123, 4) 


### 2. Taking the last 20 sessions

In [5]:
# taking all aids per session in the test set and predicting them as next steps
test_sess_aids = test.to_pandas().groupby('session')['aid'].apply(lambda x: list(x)[-20:])

In [6]:
# have a look
test_sess_aids.head(5)

session
12899779                                              [59625]
12899780           [1142000, 582732, 973453, 736515, 1142000]
12899781    [141736, 199008, 57315, 194067, 199008, 199008...
12899782    [889671, 1099390, 987399, 987399, 638410, 1072...
12899783    [255297, 1114789, 255297, 300127, 198385, 3001...
Name: aid, dtype: object

In [7]:
# creating labels
session_types = ['clicks', 'carts', 'orders']
labels_last = []

for _, aids in tqdm(test_sess_aids.iteritems()):
    for types in session_types:
        labels_last.append(' '.join([str(a) for a in aids]))

1671803it [00:03, 456334.64it/s]


In [8]:
# have a look
#labels_last

### 3. Load the sample submission csv file

In [9]:
# loading Sample Submission
sample_submission = pd.read_csv("../data/sample_submission.csv")
sample_submission.head()

,session_type,labels
0,12899779_clicks,129004 126836 118524
1,12899779_carts,129004 126836 118524
2,12899779_orders,129004 126836 118524
3,12899780_clicks,129004 126836 118524
4,12899780_carts,129004 126836 118524


### 4. Exchange the labels and save

In [10]:
# Make the submission
sample_submission["labels"] = labels_last
sample_submission.to_csv("../data/submission_last_20.csv", index=False)

### 5. Submitting to Kaggle
Recall of 0.46409

Inspiration:
https://www.kaggle.com/code/andradaolteanu/otto-i-was-warned-this-one-is-complicated